# Spitzer Pipeline from Scratch

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%autoreload

In [ ]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
rcParams["figure.dpi"] = 100

from celerite import plot_setup
plot_setup.setup(auto=False)

from pylab import *

import batman
import emcee3, corner
from sklearn.externals import joblib

from astropy.io import fits
from glob import glob

from tqdm import tqdm_notebook
emcee3.samplers.tqdm = tqdm_notebook

from scipy.optimize import leastsq, minimize

from scipy.signal import medfilt
from scipy.stats import binned_statistic

**Load Saved State**

In [ ]:
pwdSptzr  = '~/Research/Planets/WASP52/analysis/'
saveDir   = 'SaveFiles/SaveState/' 
saveName  = 'save_state_r61517056_2457927.95867_FULL.pickle.save'
# glob(saveDir + '*save_state*FULL*')

In [ ]:
load_pickle_dict = joblib.load(saveDir + saveName)# + 'FULL' + '.pickle.save')
for key in load_pickle_dict.keys():
    exec(key + "= load_pickle_dict['"+key+"']", globals(), locals())

**open the files**

In [ ]:
dataDir = '~/Research/Planets/WASP52/data/'

AORNumber   = 'r61517056'
channel     = 'ch1'
fitsFileDir = 'raw/' + AORNumber + '/' + channel + '/bcd/'
# fitsFileDir = 'raw/r51831552/ch1/bcd/'

fitsfiles = glob(dataDir + fitsFileDir + '*bcd.fits')
fitsfiles

In [ ]:
testfits    = fits.open(fitsfiles[0])[0]
testheader  = testfits.header
testheader

**Create function to loop over fits files and allocate necessary data**

In [ ]:
def spitzer_load_fits_file(fitsFilenames, outputUnits='electrons'):
    sec2day        = 1/(24*3600)
    nFramesPerFile = 64
    nSlopeFiles    = len(fitsFilenames)
    testfits       = fits.open(fitsFilenames[0])[0]
    testheader     = testfits.header
    
    bcd_shape      = testfits.data[0].shape
    
    nFrames        = nSlopeFiles * nFramesPerFile
    imageCube      = np.zeros((nFrames, bcd_shape[0], bcd_shape[1]))
    noiseCube      = np.zeros((nFrames, bcd_shape[0], bcd_shape[1]))
    timeCube       = np.zeros(nFrames)
    
    del testfits
    
    # Converts DN/s to microJy per pixel
    #   1) expTime * gain / fluxConv converts MJ/sr to electrons
    #   2) as2sr * MJ2mJ * testheader['PXSCAL1'] * testheader['PXSCAL2'] converts MJ/sr to muJ/pixel
    if outputUnits == 'electrons':
        fluxConv  = testheader['FLUXCONV']
        expTime   = testheader['EXPTIME']
        gain      = testheader['GAIN']
        fluxConversion = expTime*gain / fluxConv
    elif outputUnits == 'muJ_per_Pixel':
        as2sr = arcsec**2.0 # steradians per square arcsecond
        MJ2mJ = 1e12        # mircro-Janskeys per Mega-Jansky
        fluxConversion = abs(as2sr * MJ2mJ * testheader['PXSCAL1'] * testheader['PXSCAL2']) # converts MJ/
    else:
        raise Exception("`outputUnits` must be either 'electrons' or 'muJ_per_Pixel'")
    
    for kfile, fname in tqdm_notebook(enumerate(fitsFilenames), \
                                      desc='Spitzer Load File', leave = False, total=nSlopeFiles):
        bcdNow  = fits.open(fname)
        # buncNow = fits.open(fname[:-len(filetype)] + 'bunc.fits')
        buncNow = fits.open(fname.replace('bcd.fits', 'bunc.fits'))
        
        for iframe in range(nFramesPerFile):
            timeCube[kfile * nFramesPerFile + iframe]  = bcdNow[0].header['BMJD_OBS'] \
                            + iframe * float(bcdNow[0].header['FRAMTIME']) * sec2day
            
            imageCube[kfile * nFramesPerFile + iframe] = bcdNow[0].data[iframe]  * fluxConversion
            noiseCube[kfile * nFramesPerFile + iframe] = buncNow[0].data[iframe] * fluxConversion
        
        del bcdNow[0].data
        bcdNow.close()
        del bcdNow
        
        del buncNow[0].data
        buncNow.close()
        del buncNow
    
    return timeCube, imageCube, noiseCube

**Call Spitzer Load Fits Function**

In [ ]:
timeCube, imageCube, noiseCube = spitzer_load_fits_file(fitsfiles)

**Compute Initial Background Estimates**

This is necessary for our initial condition for centroids

In [ ]:
nFrames           = timeCube.size
median_background = median(imageCube, axis=(1,2))

**Examine the Initial Background Measurements**

In [ ]:
istart, iskip = 64, 64
plot(median_background, '.', alpha=0.02);
istart, iskip = 0, 64
plot(np.arange(median_background.size)[istart::iskip], median_background[istart::iskip], '.', alpha=0.2);
istart, iskip = 63, 64
plot(np.arange(median_background.size)[istart::iskip], median_background[istart::iskip], '.', alpha=0.2);
plt.ylim(25,160)

In [ ]:
istart, iskip = 64, 64
for istart in range(64):
    plot(np.arange(median_background.size)[istart::iskip], median_background[istart::iskip] - median(median_background[istart::iskip]), '.', alpha=0.2);
# plot(np.arange(median_background.size)[0::64], median_background[0::64], '.', alpha=0.02);
ylim(-5,5)

**Compute Flux Weighted Centroid**

This provides an initial condition for centroids

In [ ]:
def compute_flux_weighted_centroid(imageCube, yxguess, skybg, subSize=7):
    '''
        Flux-weighted centroiding (Knutson et al. 2008)
        xpos and ypos are the rounded pixel positions of the star
    '''
    
    y,x = 0,1
    
    ypos,xpos= yxguess
    ## extract a box around the star:
    
    ylower   = int(ypos-subSize)
    yupper   = int(ypos+subSize+1)
    xlower   = int(xpos-subSize)
    xupper   = int(xpos+subSize+1)
    
    nFrames  = imageCube.shape[0]
    
    flux_weighted_centroids = np.zeros((nFrames, 2))
    
    yrng  = arange(2*subSize+1)
    xrng  = arange(2*subSize+1)
    
    for kf in tqdm_notebook(range(nFrames)):
        subImage = imageCube[kf][ylower:yupper, xlower:xupper].copy()#.transpose()
        
        ## add up the flux along x and y
        yflux = (subImage - skybg[kf]).sum(axis=y)
        xflux = (subImage - skybg[kf]).sum(axis=x)
        
        ## get the flux weighted average position:
        ypeak = sum(yflux * yrng) / sum(yflux) + (ypos - float(subSize))
        xpeak = sum(xflux * xrng) / sum(xflux) + (xpos - float(subSize))
        
        flux_weighted_centroids[kf] = ypeak, xpeak
    
    return flux_weighted_centroids

In [ ]:
subSize = 7
flux_weighted_centroids = compute_flux_weighted_centroid(imageCube, [15.,15.], median_background, subSize=subSize)

**Examine the Flux Weighted Centroids**

In [ ]:
y,x = 0,1
plot(flux_weighted_centroids[:,y], '.', alpha=0.05);
plot(flux_weighted_centroids[:,x], '.', alpha=0.05);
ylim(14,16);

**Compute Gaussian Centroid**

This provides an initial condition for centroids

In [ ]:
def gaussian2D(params):
    height, offset, ycenter, xcenter, ywidth, xwidth = params
    return lambda yy,xx: height*exp(-0.5*(((ycenter-yy)/ywidth)**2. + ((xcenter-xx)/xwidth)**2.)) + offset

def mle_neg_log_like(params, image, noise, yinds, xinds):
    model = gaussian2D(params)(yinds, xinds)
    return (((image - model) / noise)**2.).sum()

def lsq_neg_log_like(params, image, noise, yinds, xinds):
    model = gaussian2D(params)(yinds, xinds)
    return (((image - model) / noise)**2.).ravel()

def fit_gauss(image, noise, initParams, yinds, xinds, method='lsq'):
    if method == 'lsq' or method == 'both':
        soln = leastsq(lsq_neg_log_like, initParams, args=(image, noise, yinds, xinds))[0]
    if method == 'mle':
        soln = minimize(mle_neg_log_like, initParams, method="L-BFGS-B", args=(image, noise, yinds, xinds))['x']
    if method == 'both':
        soln= minimize(mle_neg_log_like, soln, method="L-BFGS-B", args=(image, noise, yinds, xinds))['x']
    
    return soln

In [ ]:
def compute_gaussian_centroids(imageCube, noiseCube, initCentroids, skybgs, subSize=7, method='lsq'):
    
    y,x = 0,1
    
    nFrames  = imageCube.shape[0]
    
    if np.linalg.matrix_rank(initCentroids) == 1:
        ypos, xpos    = initCentroids
        initCentroids = np.zeros((nFrames,2)) + initCentroids
    
    if np.linalg.matrix_rank(initCentroids) == 2:
        ypos, xpos    = median(initCentroids,axis=0)
    
    yinds, xinds = np.indices((imageCube[0].shape))
    
    ypos     = np.int(np.round(ypos))
    xpos     = np.int(np.round(xpos))
    
    ylower   = int(ypos-subSize)
    yupper   = int(ypos+subSize+1)
    xlower   = int(xpos-subSize)
    xupper   = int(xpos+subSize+1)
    
    subYinds    = yinds[ylower:yupper, xlower:xupper]
    subXinds    = xinds[ylower:yupper, xlower:xupper]
    
    gaussian_centroids = np.zeros((nFrames, 6))
    
    for kf in tqdm_notebook(range(nFrames)):
        subImage    = imageCube[kf][ylower:yupper, xlower:xupper].copy()#.transpose()
        subNoise    = noiseCube[kf][ylower:yupper, xlower:xupper].copy()#.transpose()
        
        initHeight  = subImage.max()
        initOffset  = skybgs[kf]
        initYcenter = initCentroids[kf][y]
        initXcenter = initCentroids[kf][x]
        initYwidth  = 0.85
        initXwidth  = 0.85
        initParams  = [initHeight, initOffset, initYcenter, initXcenter, initYwidth, initXwidth]
        
        gaussian_centroids[kf] = fit_gauss(subImage - skybgs[kf], subNoise, initParams, subYinds, subXinds, method)
    
    return gaussian_centroids

In [ ]:
gaussian_centroids_lsq = compute_gaussian_centroids(imageCube, noiseCube, [15.,15.], median_background, method='lsq')

**Examine the Gaussian Centroids**

In [ ]:
y,x = 0,1
plot(abs(gaussian_centroids_lsq.T[y+2]), '.', color='blue', alpha=1.0, label='y-centers')
plot(abs(gaussian_centroids_lsq.T[x+2]), '.', color='orange', alpha=1.0, label='x-centers')
legend(loc=0);
# ylim(14.8, 15.1);

**Compute Aperture Photometry**

First by default, then with weight sums

In [ ]:
from photutils import CircularAperture, CircularAnnulus, EllipticalAperture
from photutils import aperture_photometry

def compute_photometry(image, noise, centroid, background, aperRad):
    ypos, xpos  = centroid
    flux_aper   = CircularAperture([xpos, ypos], aperRad)
    
    flux      = aperture_photometry(image - background, flux_aper         , \
                                    error=noise, mask=None       , \
                                    method='exact', subpixels=5 , \
                                    unit=None, wcs=None)['aperture_sum']
    
    return flux.data

def compute_photometry_series(imageCube, noiseCube, centroids, backgrounds, aperRads):
    nFrames = imageCube.shape[0]
    
    if np.rank(aperRads) == 0:
        aperRads = aperRads*np.ones(nFrames)
    
    phots     = zeros(nFrames)
    phots_err = zeros(nFrames)
    
    onesCube= ones(noiseCube.shape)
    for kf in tqdm_notebook(range(nFrames)):
        phots[kf]     = compute_photometry(imageCube[kf], \
                                           noiseCube[kf], \
                                           centroids.T[kf], \
                                           backgrounds[kf], \
                                           aperRads[kf])

        phots_err[kf] = compute_photometry(noiseCube[kf], \
                                           onesCube[kf] , \
                                           centroids.T[kf], \
                                           backgrounds[kf], \
                                           aperRads[kf])

    return phots, phots_err

In [ ]:
photmetry_2p5, photmetry_2p5_err = compute_photometry_series(imageCube, \
                                                             noiseCube, \
                                                             gaussian_centroids_lsq.T[2:4], \
                                                             median_background, 2.5)

**Effective PSF Width and Quadrature Widths**

In [ ]:
npix     = 7
midFrame = imageCube.shape[1]//2
lower    = midFrame - npix
upper    = midFrame + npix

image_view        = imageCube[:,lower:upper,lower:upper]
effective_widths  = image_view.sum(axis=(1,2))**2. / (image_view**2).sum(axis=(1,2))
var_rads          = sqrt(effective_widths)

In [ ]:
y_widths, x_widths= gaussian_centroids_lsq[:,4:].T
quadrature_widths = sqrt(x_widths**2 + y_widths**2)

In [ ]:
percLower, percMed, percUpper = np.percentile(var_rads, [50-68/2, 50, 50+68/2])
hist(var_rads,bins=len(var_rads)//500, alpha=0.5);
plt.axvline(percLower, ls='--', c='orange');
plt.axvline(percMed  , ls='-' , c='orange');
plt.axvline(percUpper, ls='--', c='orange');

nSig = 5
plt.xlim(percMed - nSig*(percMed-percLower), percMed + nSig*(percUpper - percMed));

nSig = 3.3
percLower, percMed, percUpper = np.percentile(nSig*quadrature_widths, [50-68/2, 50, 50+68/2])

hist(nSig*quadrature_widths,bins=len(nSig*quadrature_widths)//500, alpha=0.5);
plt.axvline(percLower, ls='--', c='violet');
plt.axvline(percMed  , ls='-' , c='violet');
plt.axvline(percUpper, ls='--', c='violet');

**Variable Aperture Photometry**

In [ ]:
photmetry_beta, photmetry_beta_err = compute_photometry_series(imageCube, \
                                                               noiseCube, \
                                                               gaussian_centroids_lsq.T[2:4], \
                                                               median_background, var_rads)

In [ ]:
nSig = 4.25
photmetry_qwidth, photmetry_qwidth_err = compute_photometry_series(imageCube, \
                                                               noiseCube, \
                                                               gaussian_centroids_lsq.T[2:4], \
                                                               median_background, nSig*quadrature_widths)

**Examine Binned Data for Correlation Visualization**

In [ ]:
def bin_array(arr, uncs = None,  binsize=100, KeepTheChange = False):
    '''
        Given nSize = size(time), nCols = binsize, nRows = nSize / nCols

            this function reshapes the 1D array into a new 2D array of dimension
                nCols x nRows or binsize x (nSize / nCols)

            after which, the function collapses the array into a new 1D array by taking the median

        Because most input arrays cannot be subdivided into an even number of subarrays of size `binsize`
            we actually first slice the array into a 1D array of size `nRows*binsize`.

            The mean of the remaining elements from the input array is then taken as the final element
                in the output array

    '''
    nSize   = arr.size
    nCols   = np.int(nSize / binsize)
    nRows   = binsize

    EqSize  = nRows*nCols
    #LftOvers= nSize - EqSize

    useArr  = arr[:EqSize].copy()   # this array can be subdivided evenly
    #frstElt = np.median(arr[:LftOvers/2])   # mean of the remaining elements no able to be reshaped evenly
    #lastElt = np.median(arr[-LftOvers:])   # mean of the remaining elements no able to be reshaped evenly

    if uncs is not None:
        # weighted mean profile
        useUncs = uncs[:EqSize].copy()   # this array can be subdivided evenly
        binArr  = np.median((useArr / useUncs).reshape(nCols, nRows).mean(axis=1)/ useUncs.reshape(nCols, nRows))
        stdArr  = np.median((useArr / useUncs).reshape(nCols, nRows).std(axis=1) / useUncs.reshape(nCols, nRows))
        
        if KeepTheChange:
            SpareArr    = arr[EqSize:].copy()
            SpareUncs   = uncs[EqSize:].copy()

            binTC       = np.median((SpareArr / SpareUncs)) / np.median(SpareUncs.reshape(nCols, nRows))
            stdTC       = np.median((SpareArr / SpareUncs)) / np.median(SpareUncs.reshape(nCols, nRows))

            binArr  = np.concatenate((binArr, [binTC]))
            stdArr  = np.concatenate((stdArr, [stdTC]))
    else:
        # standard mean profile
        binArr  = np.mean(useArr.reshape(nCols, nRows),axis=1)
        stdArr  = np.std(useArr.reshape(nCols, nRows),axis=1) / sqrt(nSize)
        
        if KeepTheChange:
            SpareArr    = arr[EqSize:].copy()
            binTC       = np.median(SpareArr)
            stdTC       = np.std(SpareArr)

            binArr  = np.concatenate((binArr, [binTC]))
            stdArr  = np.concatenate((stdArr, [stdTC]))

    return binArr, stdArr

In [ ]:
nbins   = 120
binsize = nFrames // nbins
bin_flux, bin_flux_err = bin_array(photmetry_2p5, uncs = None,  binsize=binsize, KeepTheChange = False)
bin_err, bin_err_err   = bin_array(photmetry_2p5_err, uncs = None,  binsize=binsize, KeepTheChange = False) 
bin_time, bin_time_err = bin_array(timeCube, uncs = None,  binsize=binsize, KeepTheChange = False)
# bin_flux = binned_statistic(timeCube, photmetry_2p5[0], statistic='mean', bins=nbins, range=None)
# bin_err  = binned_statistic(timeCube, photmetry_2p5_err[0], statistic='mean', bins=nbins, range=None)

# timebin  = bin_flux.bin_edges[1:] - median(diff(bin_flux.bin_edges))
# fluxbin  = bin_flux.statistic
# errbin   = bin_err.statistic

In [ ]:
plot(timeCube - timeCube.min(), photmetry_2p5, '.', alpha=1)
errorbar(bin_time - timeCube.min(), bin_flux, bin_flux_err, fmt='o');
ylim(39000,42000);

In [ ]:
gcY          = gaussian_centroids_lsq.T[y+2].copy()
gcX          = gaussian_centroids_lsq.T[x+2].copy()

Ysort_inds   = np.argsort(gaussian_centroids_lsq.T[y+2])
flux_Ysorted = photmetry_2p5[Ysort_inds]
gcY_Ysorted  = gaussian_centroids_lsq.T[y+2][Ysort_inds]

Xsort_inds   = np.argsort(gaussian_centroids_lsq.T[x+2])
flux_Xsorted = photmetry_2p5[Xsort_inds]

gcX_Xsorted  = gaussian_centroids_lsq.T[x+2][Xsort_inds]

bin_fluxY, bin_fluxY_err = bin_array(flux_Ysorted, uncs = None,  binsize=binsize, KeepTheChange = False)
bin_fluxX, bin_fluxX_err = bin_array(flux_Xsorted, uncs = None,  binsize=binsize, KeepTheChange = False)
bin_gcY, bin_gcYerr    = bin_array(gcY_Ysorted, uncs = None,  binsize=binsize, KeepTheChange = False)
bin_gcX, bin_gcXerr    = bin_array(gcX_Xsorted, uncs = None,  binsize=binsize, KeepTheChange = False)

# plot(gaussian_centroids_lsq.T[2], photmetry_2p5[0], '.', alpha=0.2);
plot(bin_gcY-(median(gaussian_centroids_lsq.T[y+2])), bin_fluxY,'o',alpha=0.5,mew=0);
plot(bin_gcX-(median(gaussian_centroids_lsq.T[x+2])), bin_fluxX,'o',alpha=0.5,mew=0);
# ylim(7700,8000);
# xlim(14.9,15.0);

**BATMAN Transit/Eclipse Model**

In [ ]:
def b2inc(b, aRs, e = 0, w = 0):
    #convert_b_to_inc
    if e == 0:
        return np.arccos(np.abs(b / aRs))
    elif w == 0:
        return np.arccos(np.abs(b / aRs / (1-e*e)))
    else:
        return np.arccos(np.abs(b / aRs / (1-e*e) * (1 - e*np.sin(w))))


def deltaphase_eclipse(ecc, omega, omegatype = 'RV'):
    from numpy import cos, pi, abs
    return 0.5*( 1 + (4. / pi) * ecc * cos(omega))

In [ ]:
b2inc(88.99/180*pi, 14.64, 0.26493, (360-162.149)/180*pi)

In [ ]:
h11Per       = 4.88782433
h11t0        = 2454957.812464 - 2454833.0
h11Inc       = 88.99
h11ApRs      = 14.64
h11RpRs      = 0.05856
h11Ecc       = 0.26493
h11Omega     = -162.149
h11u1        = 0.646
h11u2        = 0.048

def batman_wrapper_raw(init_params, times, ldtype='quadratic', transitType='primary'):
    
    period, tcenter, inc, aprs, rprs, ecc, omega, u1, u2 = init_params
    
    bm_params           = batman.TransitParams() # object to store transit parameters
    
    bm_params.per       = period  # orbital period
    bm_params.t0        = tcenter # time of inferior conjunction
    bm_params.inc       = inc     # inclunaition in degrees
    bm_params.a         = aprs    # semi-major axis (in units of stellar radii)
    bm_params.rp        = rprs    # planet radius (in units of stellar radii)
    bm_params.ecc       = ecc     # eccentricity
    bm_params.w         = omega   # longitude of periastron (in degrees)
    bm_params.limb_dark = ldtype              # limb darkening model # NEED TO FIX THIS
    bm_params.u         = [u1, u2]                  # limb darkening coefficients # NEED TO FIX THIS
    
    m_eclipse = batman.TransitModel(bm_params, times, transittype=transittype)    # initializes model
    
    return m_eclipse

def loglikehood(params, uni_prior, times, flux, fluxerr):
    model = batman_wrapper_raw(params, times)
    chisq = ((flux - model)/fluxerr)**2.
    return -0.5*chisq.sum() # + lambda*abs(params).sum() # + lambda*np.sqrt((params**2).sum())

def logPrior(params, uni_prior, times, flux, fluxerr):
    for kp, lower, upper in enumerate(uni_prior):
        if params[kp] < lower or params[k] > upper:
            return -np.inf
        return 0.0

def logPosterior(params, uni_prior, times, flux, fluxerr):
    logPriorNow = logPrior(params, uni_prior, times, flux, fluxerr)
    logLikeLNow = loglikehood(params, uni_prior, times, flux, fluxerr)
    return logLikeLNow + logPriorNow

def neg_logprobability(params, uni_prior, times, flux, fluxerr):
    return -2*logPosterior(params, uni_prior, times, flux, fluxerr)

periodIn    = h11Per
tcenterIn   = h11t0
incIn       = h11Inc
aprsIn      = h11ApRs
rprsIn      = h11RpRs
eccIn       = h11Ecc
omegaIn     = h11Omega
u1In        = h11u1
u2In        = h11u2

# Initial Parameters
initParams = [periodIn, tcenterIn, incIn, aprsIn, rprsIn, eccIn, omegaIn, u1In, u2In]

# Frozen Prior
uniPrior = [
            [periodIn,periodIn],
            [tcenterIn, tcenterIn],
            [incIn, incIn],
            [aprsIn, aprsIn],
            [rprsIn, rprsIn],
            [eccIn,eccIn],
            [omegaIn,omegaIn],
            [u1In,u1In],
            [u2In,u2In]
           ]

# Partial UnFrozen Prior
uniPrior = np.array([
            [periodIn,periodIn],
            [tcenterIn-0.1, tcenterIn+0.1],
            [80., 90.],
            [10, 20],
            [0.01, 0.1],
            [eccIn,eccIn],
            [omegaIn,omegaIn],
            [0.6,0.7],
            [0.0,0.1]
           ])

minimize(neg_logprobability, initParams, args=(uniPrior, timesSlice3, fluxSlice3, ferrSlice3))

In [ ]:
for k, (l,u) in enumerate(uniPrior):
    print(k,l,u)

In [ ]:
def batman_wrapper(params_in, times, ldtype = 'uniform', transittype="primary"):
    '''
        INPUT:
            params_in : 1D numpy array with parameters to be arranged into 
    '''
    iPeriod, iTCenter, iBImpact, iRsAp, iEdepth, iTdepth, iEcc, iOmega  = range(len(params_in))
    
    bm_params           = batman.TransitParams() # object to store transit parameters
    
    bm_params.per       = params_in[iPeriod]  # orbital period
    bm_params.t0        = params_in[iTCenter] # time of inferior conjunction
    bm_params.bImpact   = params_in[iBImpact] # b, impact parameter
    bm_params.r_a       = params_in[iRsAp]    # 
    bm_params.a         = 1.0 / bm_params.r_a # semi-major axis (in units of stellar radii)
    bm_params.fp        = params_in[iEdepth]  # 
    bm_params.tdepth    = params_in[iTdepth]  # from Fraine et al. 2014s
    bm_params.rp        = sqrt(params_in[iTdepth]) # planet radius (in units of stellar radii)
    bm_params.ecc       = params_in[iEcc]     # eccentricity
    bm_params.w         = params_in[iOmega]   # longitude of periastron (in degrees)
    bm_params.inc       = b2inc(bm_params.bImpact, bm_params.a, bm_params.ecc, bm_params.w)*180/pi # orbital inclination (in degrees)
    bm_params.limb_dark = ldtype              # limb darkening model # NEED TO FIX THIS
    bm_params.u         = []                  # limb darkening coefficients # NEED TO FIX THIS
    # print(bm_params.tdepth)
    bm_params.delta_phase = deltaphase_eclipse(bm_params.ecc, bm_params.w)
    bm_params.t_secondary = bm_params.t0 + bm_params.per*bm_params.delta_phase
    
    m_eclipse = batman.TransitModel(bm_params, times, transittype=transittype)    # initializes model
    
    return m_eclipse.light_curve(bm_params)

In [ ]:
from exoparams import PlanetParams

wasp52_par = PlanetParams('WASP-52 b')

iPeriod   = wasp52_par.per.value
iTCenter  = wasp52_par.t0.value-2400000.5
iBImpact  = wasp52_par.b.value
iRsAp     = 1.0/wasp52_par.ar.value
iEdepth   = 100/1e6
iTdepth   = wasp52_par.depth.value
iEcc      = wasp52_par.ecc.value
iOmega    = wasp52_par.om.value

initParams_in = np.array([iPeriod, iTCenter, iBImpact, iRsAp, iEdepth, iTdepth, iEcc, iOmega])
plot(timeCube, batman_wrapper(initParams_in, timeCube, transittype="primary"));
plot(timeCube, batman_wrapper(initParams_in, timeCube, transittype="secondary"));

**Gausssian Kernel Regression**

In [ ]:
def find_nbr_qhull(xpos, ypos, npix, sm_num = 100, a = 1.0, b = 1.0, c = 1.0, print_space = 10000.):
    '''
        Python Implimentation of N. Lewis method, described in Lewis etal 2012, Knutson etal 2012

        Taken from N. Lewis IDL code:

            Construct a 3D surface (or 2D if only using x and y) from the data
            using the qhull.pro routine.  Save the connectivity information for
            each data point that was used to construct the Delaunay triangles (DT)
            that form the grid.  The connectivity information allows us to only
            deal with a sub set of data points in determining nearest neighbors
            that are either directly connected to the point of interest or
            connected through a neighboring point

        Python Version:

        J. Fraine    first edition, direct translation from IDL 12.05.12
    '''
    from scipy.spatial import cKDTree
    #The surface fitting performs better if the data is scattered about zero

    npix    = np.sqrt(npix)

    x0  = (xpos - np.median(xpos))/a
    y0  = (ypos - np.median(ypos))/b

    if np.sum(npix) != 0.0 and c != 0:
        np0 = (npix - np.median(npix))/c
    else:
        if np.sum(npix) == 0.0:
            print('SKIPPING Noise Pixel Sections of Gaussian Kernel because Noise Pixels are Zero')
        if c == 0:
            print('SKIPPING Noise Pixel Sections of Gaussian Kernel because c == 0')

    k            = sm_num                           # This is the number of nearest neighbors you want
    n            = x0.size                          # This is the number of data points you have
    nearest      = np.zeros((k,n),dtype=np.int64)   # This stores the nearest neighbors for each data point

    #Multiplying by 1000.0 avoids precision problems
    if npix.sum() != 0.0 and c != 0:
        kdtree  = cKDTree(np.transpose((y0*1000., x0*1000., np0*1000.)))
    else:
        kdtree  = cKDTree(np.transpose((y0*1000., x0*1000.)))

    gw  = np.zeros((k,n),dtype=np.float64) # This is the gaussian weight for each data point determined from the nearest neighbors
    
    for point in tqdm_notebook(range(n),total=n):
        ind         = kdtree.query(kdtree.data[point],sm_num+1)[1][1:]
        dx          = x0[ind] - x0[point]
        dy          = y0[ind] - y0[point]

        if npix.sum() != 0.0 and c != 0:
            dnp         = np0[ind] - np0[point]

        sigx        = np.std(dx )
        sigy        = np.std(dy )
        if npix.sum() != 0.0 and c != 0:
            signp       = np.std(dnp)
        if npix.sum() != 0.0 and c != 0:
            gw_temp     = np.exp(-dx**2./(2.0*sigx**2.)) * \
                          np.exp(-dy**2./(2.*sigy**2.))  * \
                          np.exp(-dnp**2./(2.*signp**2.))
        else:
            gw_temp     = np.exp(-dx**2./(2.0*sigx**2.)) * \
                          np.exp(-dy**2./(2.*sigy**2.))

        gw_sum      = gw_temp.sum()
        gw_sum      = gw_temp.sum()
        gw[:,point] = gw_temp/gw_sum

        if (gw_sum == 0.0) or ~np.isfinite(gw_sum):
            print('(gw_sum == 0.0) or ~isfinite(gw_temp))')
            
        nearest[:,point]  = ind

    return gw.transpose(), nearest.transpose() # nearest  == nbr_ind.transpose()

In [ ]:
from statsmodels.robust import scale
def clipOutliers(array, nBins=101, nSig=10):
        if not (nBins % 2):
            print('nBins must be odd; `clipOutliers` is adding 1')
            nBins += 1
        
        medfilt_array   = medfilt(array, nBins)
        mad_array       = scale.mad(array)
        outliers        = abs(array - medfilt_array) > nSig * mad_array
        array[outliers] = medfilt_array[outliers]
        
        return array

In [ ]:
def decorrelate_gk(times, phots, params, gk, nbr_ind, transittype="primary"):
    '''
        Compute the Gaussian weighted fuction to decorelate photometry using the Gaussian kernel `gk`
    '''
    nbr_ind = np.array(nbr_ind, int).copy()
    tmodel  = batman_wrapper(params, times, transittype="primary")
    return np.sum((phots / tmodel)[nbr_ind]*gk,axis=1)

def decorrelate_gk_functionless(phots, gk, nbr_ind):
    '''
        form gaussian weighting fuction to decorelate photometry using a gaussian kernel `gk`
    '''
    nbr_ind = np.array(nbr_ind, int).copy()
    return np.sum(phots[nbr_ind]*gk,axis=1)

def gk_weighting_wrapper(gk_params):
    times, flux, gk, nbr = gk_params
    return lambda transit_params: decorrelate_gk(times, flux, transit_params, gk, nbr)

In [ ]:
y,x = 0,1
xpos = clipOutliers(gaussian_centroids_lsq[:,x+2].copy())
ypos = clipOutliers(gaussian_centroids_lsq[:,y+2].copy())
npix1= clipOutliers(var_rads.copy())
npix2= clipOutliers(4.25*quadrature_widths.copy())

In [ ]:
gw1, nbr1 = find_nbr_qhull(xpos, ypos, npix1, sm_num=100, a=1.0, b=1.0, c=1.0)
gw2, nbr2 = find_nbr_qhull(xpos, ypos, npix2, sm_num=100, a=1.0, b=1.0, c=1.0)

**Fix NaN Outliers**

In [ ]:
if len(where(np.isnan(gw2))[0]):
    ibad = where(np.isnan(gw2))
    gw2[ibad] = 0.5*(gw2[ibad-1]+gw2[ibad+1])

**Compute All Necessary Noise Models**

In [ ]:
log(std(np.diff(clipOutliers(noise_model11) / median(clipOutliers(noise_model11))))*1e3)

In [ ]:
noise_model11 = decorrelate_gk(timeCube, photmetry_2p5   , initParams_in, gw1, nbr1, transittype="primary")
noise_model12 = decorrelate_gk(timeCube, photmetry_beta  , initParams_in, gw1, nbr1, transittype="primary")
noise_model13 = decorrelate_gk(timeCube, photmetry_qwidth, initParams_in, gw1, nbr1, transittype="primary")
noise_model21 = decorrelate_gk(timeCube, photmetry_2p5   , initParams_in, gw2, nbr2, transittype="primary")
noise_model22 = decorrelate_gk(timeCube, photmetry_beta  , initParams_in, gw2, nbr2, transittype="primary")
noise_model23 = decorrelate_gk(timeCube, photmetry_qwidth, initParams_in, gw2, nbr2, transittype="primary")

**Save State**

In [ ]:
from datetime import datetime

**Examine the Effect of Each Noise Model on the Data**

In [ ]:
timeCube.min(), initParams_in[1]+0.05

In [ ]:
plot(timeCube - timeCube.mean(), photmetry_2p5/median(photmetry_2p5),'.')
xlim(-.03,.05)
ylim(0.95,1.025)

In [ ]:
paramsTest = np.copy(initParams_in)
plot(timeCube, photmetry_2p5/median(photmetry_2p5),'.')

paramsTest[1] = tcenter_range[1]#initParams_in[1]+0.05
noise_and_trans_model = gk_noise_and_transit_model(timeCube, photmetry_2p5, paramsTest, gw1, nbr1, transittypeIn="primary")
plot(timeCube, noise_and_trans_model / median(noise_and_trans_model),'.')

paramsTest[1] = tcenter_range[0]#initParams_in[1]+0.05
noise_and_trans_model = gk_noise_and_transit_model(timeCube, photmetry_2p5, paramsTest, gw1, nbr1, transittypeIn="primary")
plot(timeCube, noise_and_trans_model / median(noise_and_trans_model),'.')

ylim(.9,1.1);

In [ ]:
subplot(211)
plot(timeCube,photmetry_2p5 / median(photmetry_2p5),'.');
plot(timeCube,noise_model11 / median(photmetry_2p5),'.');
plot(timeCube, noise_and_trans_model,'.');
ylim(.9,1.1)
subplot(212)
plot(timeCube, photmetry_2p5 / noise_model11,'.');
ylim(.9,1.1)
subplot(212)
print(std(photmetry_2p5 / median(photmetry_2p5)), std(noise_model11 / median(photmetry_2p5)), std(photmetry_2p5 / noise_model11))
plot(timeCube, batman_wrapper(initParams_in, timeCube, transittype="primary"));

In [ ]:
subplot(211)
plot(timeCube,photmetry_2p5 / median(photmetry_2p5),'.');
plot(timeCube,noise_model11 / median(photmetry_2p5),'.')
# ylim(.9,1.1)
ylim(.98,1.02)
plt.xlim(.39+5.7679e4,.4+5.7679e4)
subplot(212)
plot(timeCube, photmetry_2p5 / noise_model11,'.');
ylim(.9,1.1)
subplot(212)
print(std(photmetry_2p5 / median(photmetry_2p5)), std(noise_model11 / median(photmetry_2p5)), std(photmetry_2p5 / noise_model11))
plot(timeCube, batman_wrapper(initParams_in, timeCube, transittype="primary"));
ylim(.98,1.02)
plt.xlim(.39+5.7679e4,.4+5.7679e4)

In [ ]:
bin_fluxY, bin_fluxY_err = bin_array(photmetry_2p5 / noise_model11, uncs = None,  binsize=binsize, KeepTheChange = False)
bin_time, bin_time_err   = bin_array(timeCube, uncs = None,  binsize=binsize, KeepTheChange = False)

# plot(gaussian_centroids_lsq.T[2], photmetry_2p5[0], '.', alpha=0.2);
# plot(bin_gcY-(median(gaussian_centroids_lsq.T[y+2])), bin_fluxY,'o',alpha=0.5,mew=0);
# plot(bin_gcX-(median(gaussian_centroids_lsq.T[x+2])), bin_fluxX,'o',alpha=0.5,mew=0);
# ylim(7700,8000);
# xlim(14.9,15.0);

**Optimzation with Negative Log Likelihood**

In [ ]:
BIGNUM        = 1e10

period_range  = 0.0, BIGNUM
tcenter_range = iTCenter - 0.05, iTCenter + 0.05
bimpact_range = -1.0, 1.0
RsAp_range    = 0.0, BIGNUM
edepth_range  = 0.0, 1.0
tdepth_range  = 0.0, 0.05
ecc_range     = 0.0, 1.0
omega_range   = 0.0, 360.

params_dict = dict( Period   = dict(value=iPeriod  , fit=False, priorRange=period_range , priorType='uniform', index=0),
                    TCenter  = dict(value=iTCenter , fit=True , priorRange=tcenter_range, priorType='uniform', index=1),
                    BImpact  = dict(value=iBImpact , fit=False, priorRange=bimpact_range, priorType='uniform', index=2),
                    RsAp     = dict(value=iRsAp    , fit=False, priorRange=RsAp_range   , priorType='uniform', index=3),
                    Edepth   = dict(value=iEdepth  , fit=False , priorRange=edepth_range , priorType='uniform', index=4),
                    Tdepth   = dict(value=iTdepth  , fit=True , priorRange=tdepth_range , priorType='uniform', index=5),
                    Ecc      = dict(value=iEcc     , fit=False, priorRange=ecc_range    , priorType='uniform', index=6),
                    Omega    = dict(value=iOmega   , fit=False, priorRange=omega_range  , priorType='uniform', index=7))

In [ ]:
from pandas import DataFrame
DataFrame(params_dict)

In [ ]:
%autoreload

In [ ]:
def gk_noise_and_transit_model(timesIn, photsIn, paramsIn, gkIn, nbr_indIn, transittypeIn="primary"):
    '''
        Compute the Gaussian weighted fuction to decorelate photometry using the Gaussian kernel `gk`
    '''
    nbr_ind = np.array(nbr_indIn, int).copy()
    tmodel  = batman_wrapper(paramsIn, timesIn, transittype=transittypeIn)
    return (np.sum((photsIn - tmodel)[nbr_ind]*gkIn, axis=1)+1)*tmodel

In [ ]:
def log_prior_function(paramsIn, paramDictIn, timesIn, photsIn, phots_errIn, gwIn, nbrIn, transittypeIn, minTypeIn):
    sqrt2pi = sqrt(2*pi)
    logPrior  = 0.0
    # print(minTypeIn, paramsIn)
    paramKeys = list(paramDictIn.keys())
    
    idx       = 0
    paramsUse = np.zeros(len(paramKeys))
    for key in paramKeys:
        paramIndex = np.copy(paramDictIn[key]['index'])
        if not paramDictIn[key]['fit']:
            paramsUse[paramIndex] = np.copy(paramDictIn[key]['value'])
        else:
            paramsUse[paramIndex] = np.copy(paramsIn[idx])
            idx                  += 1
    
    for key in paramKeys:
        paramNow = paramsUse[paramDictIn[key]['index']]
        if paramDictIn[key]['priorType'] == 'uniform':
            lPrior, uPrior  = paramDictIn[key]['priorRange']
            isGreater = paramNow >= lPrior
            isLower   = paramNow <= uPrior
            
            if isGreater and isLower:
                pass
            else:
                if not paramDictIn[key]['fit']:
                    raise Exception('Parameter ' + key + ' has value '  + str(paramNow)     +\
                                    ', but is out of the Prior Range [' + str(lPrior) + ', '+\
                                    str(uPrior) + '] && not set to fit')
                return -np.inf
        
        if paramDictIn[key]['priorType'] == 'normal':
            cPrior, sPrior  = paramDictIn[key]['priorRange']
            logPrior += -0.5((paramNow - cPrior)/sPrior)**2. - ln(sqrt2pi*sPrior)
        
        if paramDictIn[key]['priorType'] == 'truncated_normal':
            cPrior, sPrior, nSigPrior = paramDictIn[key]['priorRange']
            if abs((cPrior - parmamNow) <= nSigPrior*sPrior):
                logPrior += -0.5((paramNow - cPrior)/sPrior)**2. - ln(sqrt2pi*sPrior)
            else:
                return -np.inf
    
    return logPrior

In [ ]:
def neg_log_likelihood_function(paramsIn, paramDictIn, timesIn, photsIn, phots_errIn, gwIn, nbrIn, transittypeIn, minTypeIn):
    
    sqrt2pi = sqrt(2*pi)
    
    paramKeys = paramKeys = list(paramDictIn.keys())
    # print(minTypeIn, paramsIn)
    idx       = 0
    paramsUse = np.zeros(len(paramKeys))
    for key in paramKeys:
        paramIndex = np.copy(paramDictIn[key]['index'])
        if not paramDictIn[key]['fit']:
            paramsUse[paramIndex] = np.copy(paramDictIn[key]['value'])
        else:
            # print(idx, paramDict[key]['index'], paramsIn, paramsUse)
            paramsUse[paramIndex] = np.copy(paramsIn[idx])
            idx                  += 1
    # print(paramsUse[[1,5]])
    model = gk_noise_and_transit_model(timesIn, photsIn, paramsUse, gwIn, nbrIn, transittypeIn=transittypeIn)
    log_likelihood = -0.5*((photsIn - model)/phots_errIn)**2. # - log(sqrt(2*pi*phots_err))
    
    if minTypeIn == 'mle':
        # log_prior = log_prior_function(paramsUse, paramDictIn, timesIn, photsIn, phots_errIn, \
        #                                gwIn, nbrIn, transittypeIn, minTypeIn)
        # if not np.isfinite(log_prior):
        #     return -np.inf
        
        return -2*log_likelihood.sum()
    
    if minTypeIn == 'lsq':
        # log_prior = log_prior_function(paramsUse, paramDictIn, timesIn, photsIn, phots_errIn, \
        #                                gwIn, nbrIn, transittypeIn, minTypeIn)
        # if not np.isfinite(log_prior):
        #     return -np.inf*np.ones(log_likelihood.size)
        
        return -2*log_likelihood
    
    if minTypeIn == 'emcee':
        # print(log_likelihood)
        return log_likelihood.sum()

In [ ]:
from scipy.optimize import minimize, leastsq

times       = timeCube.copy()
phots       = clipOutliers(photmetry_2p5.copy())
phots_err   = clipOutliers(photmetry_2p5_err.copy())
gw, nbr     = gw1.copy(), nbr1.copy()
occulttype  = 'primary' 
minType     = 'emcee'

args_mle = params_dict, times, phots, phots_err, gw, nbr, 'primary', 'mle'
args_lsq = params_dict, times, phots, phots_err, gw, nbr, 'primary', 'lsq'

pMLE = minimize(neg_log_likelihood_function, initParams_in[[1,5]], method='L-BFGS-B', args=args_mle)#, **kwargs=kwargs_mle)
pLSQ = leastsq(neg_log_likelihood_function, initParams_in[[1,5]], args=args_lsq)#, **kwargs=kwargs_lsq)

pMLE, pLSQ, initParams_in[5]

In [ ]:
times       = timeCube.copy()
phots       = clipOutliers(photmetry_2p5.copy(), nSig=8)
phots_err   = clipOutliers(photmetry_2p5_err.copy(), nSig=8)
gw, nbr     = gw1.copy(), nbr1.copy()
occulttype  = 'primary' 
minType     = 'emcee'

model     = emcee3.SimpleModel(neg_log_likelihood_function, log_prior_function, \
    args=(params_dict, times, phots, phots_err, gw1, nbr1, occulttype, minType))

freeParams = []
for key in params_dict.keys():
    if params_dict[key]['fit']:
        freeParams.append(params_dict[key]['index'])

print(freeParams)

ndim, nwalkers = len(freeParams), 100
coords = np.zeros((ndim,nwalkers))

idx = 0
for key in params_dict.keys():
    if params_dict[key]['fit']:
        initPNow = params_dict[key]['value']
        initRng  = 0.05*params_dict[key]['value']
        lPrior   = initPNow - initRng
        uPrior   = initPNow + initRng
        coords[idx]    = np.random.uniform(lPrior, uPrior, nwalkers)
        idx           += 1

# print(coords.T)

In [ ]:
model(coords=initParams_in[[1,5]])

In [ ]:
ensemble = emcee3.Ensemble(model, coords.T)

print("The first walker has the coordinates: {0}".format(ensemble[0].coords))
print("and log prior={0:.3f}, log likelihood={1:.3f}, and log probability={2:.3f}"
      .format(ensemble[0].log_prior, ensemble[0].log_likelihood, ensemble[0].log_probability))

nIters  = 100
sampler = emcee3.Sampler()
sampler.run(ensemble, nIters, progress=True);

In [ ]:
paramLabels = ["$Period$", "$tCenter$", r"$b_{impact}$", r"$\frac{Rs}{Ap}$", "$E-depth$", "$T-depth$", "$e$", "$\omega$"]

# chain = sampler.get_coords(discard=10, flat=True) # need to change indicies in plot function
chain = sampler.get_coords()
fig, axes = plt.subplots(len(freeParams), 1, sharex=True, figsize=(6, 6))
for k, nm in enumerate(np.array(paramLabels)[freeParams]):
    if len(freeParams) > 1:
        axes[k].plot(chain[:, :, k], alpha=0.3)
#         axes[k].locator_params(tight=True, nbins=6)
        axes[k].yaxis.set_label_coords(-0.15, 0.5)
        axes[k].set_ylabel(nm)
    else:
        axes.plot(chain[:, :, k], alpha=0.3)
        # axes.locator_params(tight=True, nbins=6)
        axes.yaxis.set_label_coords(-0.15, 0.5)
        axes.set_ylabel(nm)

In [ ]:
import corner

flatchain    = sampler.get_coords(discard=60, flat=True)
medianParams = median(flatchain,axis=0)
madParams    = scale.mad(flatchain,axis=0)
stdParams    = std(flatchain,axis=0)

print(medianParams*1e6, madParams*1e6, stdParams*1e6)

corner.corner(flatchain[(flatchain>0.02004)*(flatchain<0.02005)], labels=np.array(paramLabels)[freeParams], truths=medianParams);
# xlim(0.0200,0.201)
fig = gcf()
ax  = fig.get_axes()[0]
# ax.set_xlim(.02,.0201)
axvline(0.02)
axvline(0.0201);

In [ ]:
idx       = 0
paramsUse = np.zeros(len(params_dict.keys()))
for key in params_dict.keys():
    if not params_dict[key]['fit']:
        paramIndex            = params_dict[key]['index']
        paramsUse[paramIndex] = params_dict[key]['value']
    else:
        paramsUse[paramIndex] = medianParams
        idx                  += 1

transittype='primary'
# for i in np.random.randint(len(flatchain), size=50):
#     #for fp in freeParams:
#     paramsUse[idx] = flatchain[i]
#     model = gk_noise_and_transit_model(times, phots, paramsUse, gw, nbr, transittype=transittype)
#     plt.plot(timeCube, model, "orange", alpha=0.1)

# plot the data
nbins   = 120
binsize = nFrames // nbins

times_bin, times_bin_err = bin_array(times        , uncs = None,  binsize=binsize, KeepTheChange = False)
phots_bin, phots_bin_err = bin_array(phots        , uncs = None,  binsize=binsize, KeepTheChange = False)
phots_err_bin, phots_err_err = bin_array(phots_err, uncs = None,  binsize=binsize, KeepTheChange = False)

# plt.plot(times, phots, phots_err, 'k.', alpha=0.01)
plt.errorbar(times_bin, phots_bin, phots_err_bin, fmt='.',color='k')


paramsUse[idx] = medianParams
modelMed = gk_noise_and_transit_model(times, phots, paramsUse, gw, nbr, transittype=transittype)
modelMed_bin, modelMed__err = bin_array(modelMed, uncs = None,  binsize=binsize, KeepTheChange = False)

plt.plot(times_bin, modelMed_bin, 'o',color="orange", alpha=1)
plt.ylim(8000,8500)

In [ ]:
q = np.percentile(flatchain, [16, 50, 84], axis=0)
mean = q[1]
uncert = np.diff(q, axis=0).T

for i, (nm, truth) in enumerate(zip(paramLabels,medianParams)):
    display(Math(r"{0}_\mathrm{{MCMC}} = {1:.3f} _{{-{2:.3f}}}^{{+{3:.3f}}} \quad (\mathrm{{truth:}}\,{4:.3f})"
                 .format(nm, mean[i], uncert[i][0], uncert[i][1], truth)))